# * Sales Channel

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

In [2]:
''' Input parameter '''

op_dir = 'data'
op_sales_channel_file = f'sales_channel_{str_curr_dt}'

# v_year = 0
# v_month_start = 0
# v_month_end = 0
# v_date = 20250101

print(f'\nParameter input...\n')
print(f'   -> op_dir: {op_dir}')
print(f'   -> op_sales_channel_file: {op_sales_channel_file}')
# print(f'\n   -> v_year: {v_year}')
# print(f'   -> v_month_start: {v_month_start}')
# print(f'   -> v_month_end: {v_month_end}')
# print(f'\n   -> v_date: {v_date}')


Parameter input...

   -> op_dir: data
   -> op_sales_channel_file: sales_channel_20251224


## Import Transaction
-> DASH_PERFORMANCE

In [3]:
''' Execute transaction '''


# Input parameter
v_start_date = 20250101
print(f'\nParameter input...')
print(f'   -> v_start_date: {v_start_date}')

curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
print(f'\nData as of {curr_datetime}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()

query = (f"""
    SELECT /*+PARALLEL(8)*/ 
        TM_KEY_MTH, COMPANY, PRODUCT, SUB_PRODUCT, GP_SKU, SPECIAL_SKU, PROJECT_GROUP, CUST_TYPE, BU_CHANNEL, TDS_CHANNEL, TDS_SUB_CHANNEL, GROUP_TDS_SPECIAL, TDS_SPECIAL_CHANNEL, CHANNEL_GROUP, BUNDLE_TYPE
        , ORGID_H, HOP_HINT, D_CLUSTER, ORGID_G, TDS_GMD, TDS_SGMD, DISTRICT_TH, DISTRICT_EN
        , A.CCAA, O.PROVINCE_CD, O.PROVINCE_ENG, O.PROVINCE_TH
        , TARGET, ACTUAL, RUNRATE, TARGET_VALUE, ACTIVATION_VALUE, ACTIVATION_VALUE_RUNRATE
        
    FROM (
        SELECT SUBSTR(TM_KEY_DAY, 1, 6) TM_KEY_MTH
            , COMPANY, PRODUCT, SUB_PRODUCT, GP_SKU, SPECIAL_SKU, PROJECT_GROUP, CUST_TYPE, BU_CHANNEL, TDS_CHANNEL, TDS_SUB_CHANNEL, GROUP_TDS_SPECIAL, TDS_SPECIAL_CHANNEL, CHANNEL_GROUP, BUNDLE_TYPE 
            , ORGID_H, HOP_HINT, D_CLUSTER, ORGID_G, TDS_GMD, TDS_SGMD, DISTRICT_TH, DISTRICT_EN
            , SUBSTR(PARTNER_CCAATT, 1, 4) AS CCAA --, PARTNER_CCAATT, PARTNER_SCAB
            , SUM(TARGET) TARGET, SUM(ACTUAL) ACTUAL, SUM(RUNRATE) RUNRATE, SUM(TARGET_VALUE) TARGET_VALUE, SUM(ACTIVATION_VALUE) ACTIVATION_VALUE, SUM(ACTIVATION_VALUE_RUNRATE) ACTIVATION_VALUE_RUNRATE
            
        FROM CDSSHRE.DASH_PERFORMANCE
        WHERE TM_KEY_DAY >= {v_start_date}
        AND ( (SUB_PRODUCT = 'POSTPAY' AND CUST_TYPE = 'INDY')
            OR (SUB_PRODUCT IN ('PREPAY', 'INFLOW M1', 'INFLOW M2', 'AP', 'APD1') AND CUST_TYPE = 'INDY')
            OR SUB_PRODUCT = 'TOL' --AND GP_SKU = 'Consumer Service'
            OR SUB_PRODUCT IN ('TVS NOW APP', 'CMDU') )

        GROUP BY SUBSTR(TM_KEY_DAY, 1, 6) 
            , COMPANY, PRODUCT, SUB_PRODUCT, GP_SKU, SPECIAL_SKU, PROJECT_GROUP, CUST_TYPE, BU_CHANNEL, TDS_CHANNEL, TDS_SUB_CHANNEL, GROUP_TDS_SPECIAL, TDS_SPECIAL_CHANNEL, CHANNEL_GROUP, BUNDLE_TYPE 
            , ORGID_H, HOP_HINT, D_CLUSTER, ORGID_G, TDS_GMD, TDS_SGMD, DISTRICT_TH, DISTRICT_EN
            , SUBSTR(PARTNER_CCAATT, 1, 4)
    ) A
    LEFT JOIN (
        SELECT DISTINCT SUBSTR(CCAATT,1,4) AS CCAA--, DISTRICT_EN
            , SUBSTR(CCAATT,1,2) AS PROVINCE_CD, PROVINCE_ENG, PROVINCE_TH
        FROM CDSAPPO.DIM_MOOC_AREA
        WHERE TEAM_CODE <> 'ไม่ระบุ' AND REMARK <> 'Dummy'
    ) O
        ON O.CCAA = A.CCAA
""")


try:
    # Create Dataframe
    src_cur.execute(query)
    rows = src_cur.fetchall()
    src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\nDataFrame: {src_df.shape[0]} rows, {src_df.shape[1]} columns')

    # Generate CSV file
    src_df.to_csv(f'{op_dir}/{op_sales_channel_file}.csv', index=False, encoding='utf-8')
    print(f'\n   -> Generate "{op_sales_channel_file}.csv" successfully')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...
   -> v_start_date: 20250101

Data as of 2025-12-24, 15:48:18

DataFrame: 1596074 rows, 33 columns

   -> Generate "sales_channel_20251224.csv" successfully


## Review

In [4]:
src_df

,TM_KEY_MTH,COMPANY,PRODUCT,SUB_PRODUCT,GP_SKU,SPECIAL_SKU,PROJECT_GROUP,CUST_TYPE,BU_CHANNEL,TDS_CHANNEL,...,CCAA,PROVINCE_CD,PROVINCE_ENG,PROVINCE_TH,TARGET,ACTUAL,RUNRATE,TARGET_VALUE,ACTIVATION_VALUE,ACTIVATION_VALUE_RUNRATE
0,202508,TRUE,PREPAY,INFLOW M1,MASS,MASS,MASS,INDY,7-11,7-11,...,3610,36,Chaiyaphum,ชัยภูมิ,0.00,0,0.0,0.0,9775.90,9775.90
1,202502,TRUE,PREPAY,INFLOW M1,STUDENT,STUDENT,STUDENT,INDY,Non-Shop,Wholesales,...,4607,46,Kalasin,กาฬสินธุ์,0.00,0,0.0,0.0,4837.42,4837.42
2,202503,DTAC,POSTPAY,POSTPAY,Migrate,Migrate,None,INDY,DTAC Channel,Unidentified,...,7303,73,Nakhon Pathom,นครปฐม,0.00,70,70.0,0.0,25453.00,25453.00
3,202503,DTAC,PREPAY,APD1,MIGRANT,MIGRANT,None,INDY,DTAC Channel,Unidentified,...,7005,70,Ratchaburi,ราชบุรี,0.00,418,418.0,0.0,0.00,0.00
4,202503,DTAC,PREPAY,INFLOW M1,MIGRANT,MIGRANT,None,INDY,DTAC Channel,Unidentified,...,1012,10,Bangkok,กรุงเทพมหานคร,0.00,0,0.0,0.0,62341.56,62341.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1596069,202504,DTAC,PREPAY,PREPAY,MASS,MASS,None,INDY,DTAC Channel,Unidentified,...,2005,20,Chon Buri,ชลบุรี,0.00,103,103.0,0.0,0.00,0.00
1596070,202504,DTAC,PREPAY,PREPAY,MASS_MIGRANT,MASS_MIGRANT,None,INDY,DTAC Channel,Unidentified,...,3901,39,Nong Bua Lam Phu,หนองบัวลำภู,260.00,0,0.0,0.0,0.00,0.00
1596071,202504,TRUE,PREPAY,PREPAY,MASS,MASS,MASS,INDY,Non-Shop,Wholesales,...,4008,40,Khon Kaen,ขอนแก่น,111.26,0,0.0,0.0,0.00,0.00
1596072,202507,DTAC,PREPAY,INFLOW M1,MASS_MIGRANT,MASS_MIGRANT,None,INDY,DTAC Channel,Unidentified,...,3801,38,Bueng Kan,บึงกาฬ,0.00,0,0.0,0.0,0.00,0.00
